In [ ]:
import os
import io
import sys
import zipfile
import tempfile
import subprocess

import librosa
import parselmouth

import math
import numpy as np
import scipy.signal
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from typing import List, Dict, Tuple, Callable, Any

print(f'Versão do Cuda: {torch.__version__}')
print(f'Disponibilidade da GPU: {torch.cuda.is_available()}')

Versão do Cuda: 2.9.0+cu130
Disponibilidade da GPU: True


##### CNN multiclass

In [ ]:
OUTPUT_DIR = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\data'
INPUT_FILES = [
    'features_z_score.csv',
    'features_min_max_0_1.csv',
    'features_signal_norm_-1_1_z_score.csv',
    'features_signal_norm_-1_1_min_max_0_1.csv'
]

BATCH_SIZE = 32
N_EPOCHS = 200
LEARNING_RATE = 1e-3
KFOLD_SPLITS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class FeatureDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx]

class MulticlassCNN(nn.Module):
    # Pooling assimétrico
    # dict = {'ID001': [AGE, SEX,...], 'ID002': []}
    def __init__(self, input_dim: int, num_classes: int):
        super(MulticlassCNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            # Camada 1: 1 -> 32 canais. Dimensão / 2
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), 
            
            # Camada 2: 32 -> 64 canais. Dimensão / 4
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Camada 3: 64 -> 128 canais. Dimensão / 8
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Camada 4: 128 -> 256 canais. Dimensão / 16
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Camada 5: 256 -> 512 canais. Dimensão / 32
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        flatten_size = 512 * (input_dim // 32) 
        
        self.fc_layer = nn.Sequential(
            nn.Linear(flatten_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.5), 
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

def train_and_evaluate(df: pd.DataFrame, filename: str) -> Dict[str, float]:
    print(f"\n{'='*50}\nIniciando treinamento para: {filename}\n{'='*50}")

    sex_mapping = {'M': 0, 'F': 1}
    df['Sex'] = df['Sex'].map(sex_mapping)
    
    X_raw = df.drop(columns=['ID', 'Class']).values
    y_raw = df['Class'].values
    
    X = np.nan_to_num(X_raw, nan=0.0) 
    
    le = LabelEncoder()
    y = le.fit_transform(y_raw)
    
    INPUT_DIM = X.shape[1]
    NUM_CLASSES = len(np.unique(y))
    
    print(f"Dimensão da entrada (Input_dim): {INPUT_DIM}")
    print(f"Número de classes: {NUM_CLASSES}")

    skf = StratifiedKFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    fold_results = {'accuracy': [], 'f1_weighted': []}

    for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
        
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")

        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = FeatureDataset(X_train, y_train)
        val_dataset = FeatureDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        model = MulticlassCNN(INPUT_DIM, NUM_CLASSES).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(N_EPOCHS):
            model.train()
            total_loss = 0
            for features, labels in train_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            if (epoch + 1) % 10 == 0:
                print(f"Fold {fold+1}, Epoch {epoch+1}/{N_EPOCHS}, Loss: {total_loss / len(train_loader):.4f}")

        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                outputs = model(features)
                _, predicted = torch.max(outputs.data, 1)
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        acc = accuracy_score(all_labels, all_preds)
        f1_w = f1_score(all_labels, all_preds, average='weighted')
        
        fold_results['accuracy'].append(acc)
        fold_results['f1_weighted'].append(f1_w)
        
        print(f"Resultado do Fold {fold+1}: Accuracy = {acc:.4f}, F1-Weighted = {f1_w:.4f}")

    final_results = {
        'Filename': filename,
        'Mean_Accuracy': np.mean(fold_results['accuracy']),
        'Std_Accuracy': np.std(fold_results['accuracy']),
        'Mean_F1_Weighted': np.mean(fold_results['f1_weighted']),
        'Std_F1_Weighted': np.std(fold_results['f1_weighted'])
    }
    
    print(f"\nResumo Final ({filename}):")
    print(f"  Média Accuracy: {final_results['Mean_Accuracy']:.4f} (+/- {final_results['Std_Accuracy']:.4f})")
    print(f"  Média F1-W: {final_results['Mean_F1_Weighted']:.4f} (+/- {final_results['Std_F1_Weighted']:.4f})")
    
    return final_results

missing_files = [f for f in INPUT_FILES if not os.path.exists(os.path.join(OUTPUT_DIR, f))]
if missing_files:
    print(f"ERRO: Os seguintes arquivos de entrada não foram encontrados no diretório {OUTPUT_DIR}:")
    for f in missing_files:
        print(f"- {f}")
    print("\nCertifique-se de executar o script de extração e normalização primeiro.")
    sys.exit(1)

all_experiment_results = []

for filename in INPUT_FILES:
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    try:
        df = pd.read_csv(filepath)
        results = train_and_evaluate(df, filename)
        all_experiment_results.append(results)
    except Exception as e:
        print(f"Erro ao processar o arquivo {filename}: {e}", file=sys.stderr)

results_df = pd.DataFrame(all_experiment_results)
output_results_path = os.path.join(OUTPUT_DIR, 'cnn_multiclass_results.csv')
results_df.to_csv(output_results_path, index=False)

print("\n\n" + "="*70)
print("RESUMO FINAL DOS EXPERIMENTOS")
print("="*70)
print(results_df)
print(f"\nResultados consolidados salvos em: {output_results_path}")

##### One-vs-Rest

In [ ]:
OUTPUT_DIR = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\data'

INPUT_FILES = [
    'features_z_score.csv',
    'features_min_max_0_1.csv',
    'features_signal_norm_-1_1_z_score.csv',
    'features_signal_norm_-1_1_min_max_0_1.csv'
]

BATCH_SIZE = 32
N_EPOCHS = 50
LEARNING_RATE = 1e-3
KFOLD_SPLITS = 5 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_BINARY_CLASSES = 2 
NUM_CLASSES = 5
TARGET_CLASSES = [1, 2, 3, 4, 5] 

# ARQUITETURA DA CNN

class BinaryCNN(nn.Module):
    def __init__(self, input_dim: int, num_output_classes: int = 2):
        super(BinaryCNN, self).__init__()
        
        # Bloco Convolucional (5 Camadas)
        self.conv_layer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), 
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # O cálculo do tamanho da camada linear depende da dimensão de entrada
        # Garante que a dimensão seja divisível por 32
        input_dim_check = input_dim // 32
        if input_dim_check == 0:
            # Caso a dimensão seja muito pequena, use 1 como fallback ou ajuste o modelo
            print(f"AVISO: Dimensão de entrada ({input_dim}) muito pequena. Usando 1.")
            input_dim_check = 1 
            
        flatten_size = 512 * input_dim_check
        
        # Bloco Densely Connected
        self.fc_layer = nn.Sequential(
            nn.Linear(flatten_size, 1024), 
            nn.ReLU(),
            nn.Dropout(0.5), 
            nn.Linear(1024, num_output_classes) # Sempre 2 classes
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1) 
        x = self.fc_layer(x)
        return x

# DATASET

class FeatureDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, is_ovr_target: bool = False):
        self.X = torch.tensor(X, dtype=torch.float32)
        
        if is_ovr_target:
            self.y = torch.tensor(y, dtype=torch.long)
        else:
            # Rótulos originais (1-5)
            self.y = torch.tensor(y, dtype=torch.long) 

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx]

# FUNÇÃO AUXILIAR DE TREINAMENTO BINÁRIO (OvR) E COLETA DE SCORES

def train_ovr_model_and_collect_scores(
    X: np.ndarray, 
    y_binary: np.ndarray, 
    input_dim: int, 
    target_class: int,
    file_fold_seed: int
) -> Tuple[Dict[str, float], List[Tuple[np.ndarray, np.ndarray]]]:
    """Treina e avalia OvR, retornando métricas e scores de validação."""
    
    skf = StratifiedKFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42 + file_fold_seed) 
    fold_metrics = {'accuracy': [], 'f1_weighted': []}
    
    # Armazenará (indices de validação, scores de validação) para agregação
    all_validation_data = [] 

    for k_fold, (train_index, val_index) in enumerate(skf.split(X, y_binary)):
        
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y_binary[train_index], y_binary[val_index]

        # Datasets com rótulos binários (OvR)
        train_dataset = FeatureDataset(X_train, y_train, is_ovr_target=True)
        val_dataset = FeatureDataset(X_val, y_val, is_ovr_target=True)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Inicialização e Treinamento do Modelo
        model = BinaryCNN(input_dim, NUM_BINARY_CLASSES).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(N_EPOCHS):
            model.train()
            for features, labels in train_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Avaliação e Coleta de Scores
        model.eval()
        fold_preds = []
        fold_labels = []
        fold_scores = [] # Logits brutos
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                outputs = model(features)
                
                _, predicted = torch.max(outputs.data, 1)
                
                fold_preds.extend(predicted.cpu().numpy())
                fold_labels.extend(labels.cpu().numpy())
                
                # Coletar os logits (scores) para a classe positiva (índice 1)
                fold_scores.extend(outputs[:, 1].cpu().numpy()) 

        # Métricas do Fold
        acc = accuracy_score(fold_labels, fold_preds)
        f1_w = f1_score(fold_labels, fold_preds, average='weighted', zero_division=0) 
        
        fold_metrics['accuracy'].append(acc)
        fold_metrics['f1_weighted'].append(f1_w)
        
        # Armazena os resultados para a agregação OvR
        all_validation_data.append((val_index, np.array(fold_scores)))
    
    mean_metrics = {
        'Target_Class': target_class,
        'Mean_Accuracy': np.mean(fold_metrics['accuracy']),
        'Std_Accuracy': np.std(fold_metrics['accuracy']),
        'Mean_F1_Weighted': np.mean(fold_metrics['f1_weighted']),
        'Std_F1_Weighted': np.std(fold_metrics['f1_weighted'])
    }
    
    return mean_metrics, all_validation_data

# FUNÇÃO PRINCIPAL DE TREINAMENTO OVR E AGREGAÇÃO PARA 5X5

def run_ovr_training(df: pd.DataFrame, filename: str, file_index: int) -> Tuple[List[Dict], np.ndarray]:
    
    print(f"\n{'='*70}\nIniciando Pipeline One-vs-Rest (OvR) para: {filename}\n{'='*70}")

    # Pré-processamento inicial
    sex_mapping = {'M': 0, 'F': 1}
    df['Sex'] = df['Sex'].map(sex_mapping)
    
    X_raw = df.drop(columns=['ID', 'Class']).values
    y_original = df['Class'].values
    
    # Lidar com NaNs nos dados
    X = np.nan_to_num(X_raw, nan=0.0) 
    INPUT_DIM = X.shape[1]
    TOTAL_SAMPLES = len(X)
    
    print(f"  Dimensão de Features (X): {X.shape}")
    
    all_ovr_results = []
    
    # Inicializa a matriz para armazenar os scores de OvR para cada amostra (Total x Classes)
    aggregated_ovr_scores = np.zeros((TOTAL_SAMPLES, NUM_CLASSES)) 
    
    # ----------------------------------------------------
    # Loop de Treinamento OvR para cada classe (1, 2, 3, 4, 5)
    # ----------------------------------------------------
    for target_class in TARGET_CLASSES:
        level_name = f"OvR_Class_{target_class} (vs Restante)"
        print(f"\n--- {level_name} ---")
        
        # 1. Cria o target binário para o modelo OvR
        y_binary = np.where(y_original == target_class, 1, 0)
        
        # 2. Treinamento e Coleta de Scores
        results, validation_data = train_ovr_model_and_collect_scores(
            X, y_binary, INPUT_DIM, target_class, file_fold_seed=file_index + target_class
        )
        
        # 3. Consolida e armazena os resultados
        results['Filename'] = filename
        results['Classifier'] = level_name
        all_ovr_results.append(results)
        
        print(f"  Accuracy: {results['Mean_Accuracy']:.4f}, F1-W: {results['Mean_F1_Weighted']:.4f}")

        # 4. Agregação dos scores OvR
        ovr_class_index = target_class - 1 
        for val_index, scores in validation_data:
            aggregated_ovr_scores[val_index, ovr_class_index] = scores


    # ----------------------------------------------------
    # AGREGAÇÃO FINAL (5X5) PARA MATRIZ DE CONFUSÃO
    # ----------------------------------------------------
    
    # A previsão final é a classe cujo modelo OvR retornou o maior score (logit)
    # Argmax retorna o índice (0 a 4), adicionamos 1 para a classe (1 a 5)
    final_predictions_1_to_5 = np.argmax(aggregated_ovr_scores, axis=1) + 1 
    
    # Rótulos verdadeiros (1 a 5)
    true_labels_1_to_5 = y_original
    
    # Calcula a Matriz de Confusão 5x5
    conf_matrix = confusion_matrix(
        true_labels_1_to_5, 
        final_predictions_1_to_5, 
        labels=TARGET_CLASSES
    )
    
    return all_ovr_results, conf_matrix

# FUNÇÃO DE EXIBIÇÃO DA MATRIZ

def display_confusion_matrix(conf_matrix: np.ndarray, filename: str, save_dir: str = OUTPUT_DIR):
    """Formata, imprime e salva a matriz de confusão (5x5)."""
    
    print("\n" + "#"*70)
    print(f"MATRIZ DE CONFUSÃO (5x5) OVR - Agregação de K-Fold para: {filename}")
    print(f"Rótulos: [1, 2, 3, 4, 5]")
    print("#"*70)
    
    header = ["Classe Real ->"] + [f"Pred {c}" for c in TARGET_CLASSES]
    print("{:<15}".format(""), end="")
    print(" ".join(["{:>8}".format(h) for h in header[1:]]))
    print("-" * (15 + 8 * 5))
    for i in range(5):
        row_label = f"Classe {TARGET_CLASSES[i]} |"
        print("{:<15}".format(row_label), end="")
        for j in range(5):
            print("{:>8}".format(conf_matrix[i, j]), end="")
        print()
    print("#"*70)
    
    plt.figure(figsize=(7, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=TARGET_CLASSES, yticklabels=TARGET_CLASSES)
    plt.title(f'Matriz de Confusão - {filename}')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.tight_layout()
    
    save_path = os.path.join(save_dir, f'confusion_matrix_{filename.replace(".csv", "")}.png')
    plt.savefig(save_path, dpi=300)
    plt.close()
    
    print(f"Matriz de confusão salva em: {save_path}")
 
# EXECUÇÃO
missing_files = [f for f in INPUT_FILES if not os.path.exists(os.path.join(OUTPUT_DIR, f))]
if missing_files:
    print(f"AVISO: Arquivos não encontrados no diretório {OUTPUT_DIR}. O script tentará pular os ausentes:")
    for f in missing_files:
        print(f"- {f}")

all_experiment_results = []
first_conf_matrix = None
first_filename = None

for idx, filename in enumerate(INPUT_FILES):
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    if not os.path.exists(filepath):
            continue
    
    try:
        df = pd.read_csv(filepath)
        
        # O run_ovr_training retorna a lista de resultados OvR e a matriz 5x5 agregada
        results_list, conf_matrix = run_ovr_training(df, filename, file_index=idx)
        
        all_experiment_results.extend(results_list)
        
        display_confusion_matrix(conf_matrix, filename, save_dir=OUTPUT_DIR)
        
        # Armazena a primeira matriz para exibição detalhada
        if first_conf_matrix is None:
            first_conf_matrix = conf_matrix
            first_filename = filename
            
    except Exception as e:
        print(f"Erro ao processar o arquivo {filename}: {e}", file=sys.stderr)

if first_conf_matrix is not None:
    display_confusion_matrix(first_conf_matrix, first_filename, save_dir=OUTPUT_DIR)

results_df = pd.DataFrame(all_experiment_results)
if not results_df.empty:
    output_results_path = os.path.join(OUTPUT_DIR, 'cnn_ovr_results.csv')
    results_df.to_csv(output_results_path, index=False)
    
    print("\n\n" + "="*70)
    print("RESUMO FINAL DOS EXPERIMENTOS ONE-VS-REST (OvR)")
    print("="*70)
    # Exibir o resumo agrupado por arquivo e classificador
    print(results_df.sort_values(by=['Filename', 'Target_Class']))
    print(f"\nResultados consolidados salvos em: {output_results_path}")
else:
    print("\nNenhum resultado foi gerado. Verifique se os arquivos de entrada existem.")


Iniciando Pipeline One-vs-Rest (OvR) para: features_z_score.csv
  Dimensão de Features (X): (272, 186)

--- OvR_Class_1 (vs Restante) ---
  Accuracy: 0.9780, F1-W: 0.9671

--- OvR_Class_2 (vs Restante) ---
  Accuracy: 0.9046, F1-W: 0.8805

--- OvR_Class_3 (vs Restante) ---
  Accuracy: 0.6910, F1-W: 0.6759

--- OvR_Class_4 (vs Restante) ---
  Accuracy: 0.6247, F1-W: 0.6076

--- OvR_Class_5 (vs Restante) ---
  Accuracy: 0.6323, F1-W: 0.6251

######################################################################
MATRIZ DE CONFUSÃO (5x5) OVR - Agregação de K-Fold para: features_z_score.csv
Rótulos: [1, 2, 3, 4, 5]
######################################################################
                 Pred 1   Pred 2   Pred 3   Pred 4   Pred 5
-------------------------------------------------------
Classe 1 |            0       1       1       3       1
Classe 2 |            1       5      11       3       6
Classe 3 |            2       3      14      18      20
Classe 4 |            0   

In [ ]:
OUTPUT_DIR = r'C:\Users\joaov_zm1q2wh\python\icassp_challenge\data'

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_FILE_FOR_SFS = 'features_z_score.csv'
BATCH_SIZE = 32
N_EPOCHS = 50
LEARNING_RATE = 1e-3
KFOLD_SPLITS = 5 
NUM_BINARY_CLASSES = 2 
NUM_CLASSES = 5
TARGET_CLASSES = [1, 2, 3, 4, 5] 

ALL_COLUMNS = [
    'Age', 'Sex',
    'Fo_mean_Hz', 'Fhi_max_Hz', 'Flo_min_Hz', 'F0_std_Hz', 
    'Jitter_percent', 'Jitter_Abs', 'RAP', 'PPQ', 'DDP', 
    'Shimmer_local', 'Shimmer_dB', 'Shimmer_APQ3', 'Shimmer_APQ5', 
    'Shimmer_APQ11', 'Shimmer_DDA', 'NHR', 'HNR', 'RPDE', 
    'DFA', 'spread1', 'spread2', 'D2', 'PPE'
]

def get_grouped_columns(base_col: str) -> List[str]:
    """Expande uma feature base para incluir todos os grupos (A, E, I, O, U, KA, PA, TA)."""
    if base_col in ['Age', 'Sex']:
        return [base_col]
    
    # Grupos de vogais (A, E, I, O, U)
    vowels = ['phonationA', 'phonationE', 'phonationI', 'phonationO', 'phonationU']
    vowel_cols = [f"{base_col}_{v}" for v in vowels]
    
    # # Grupos de ritmos (KA, PA, TA)
    # rhythms = ['rhythmKA', 'rhythmPA', 'rhythmTA']
    # rhythm_cols = [f"{base_col}_{r}" for r in rhythms]
    
    return vowel_cols

# Define a ordem de adição dos grupos de features e remove os grupos que já estão no início do fluxo para evitar re-adição.
FEATURE_GROUPS = [get_grouped_columns(col) for col in ALL_COLUMNS]
FEATURE_GROUPS = [g for g in FEATURE_GROUPS if g[0] not in ['Age', 'Sex', 'Fo_mean_Hz_phonationA', 'Fo_mean_Hz_phonationE', 'Fo_mean_Hz_phonationI', 'Fo_mean_Hz_phonationO', 'Fo_mean_Hz_phonationU']] 

# DATASET
class FeatureDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, is_ovr_target: bool = False):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long) 

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx]

# CNN
class BinaryCNN(nn.Module):
    def __init__(self, input_dim: int, num_output_classes: int = 2):
        super(BinaryCNN, self).__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        input_dim_after_conv = input_dim // (2**2) 
        
        if input_dim_after_conv == 0:
             input_dim_after_conv = 1 
             
        flatten_size = 64 * input_dim_after_conv
        
        self.fc_layer = nn.Sequential(
            nn.Linear(flatten_size, 1024), 
            nn.ReLU(),
            nn.Dropout(0.5), 
            nn.Linear(1024, num_output_classes)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1) 
        x = self.fc_layer(x)
        return x

# TREINAMENTO
def train_ovr_model_and_collect_scores(X: np.ndarray, y_binary: np.ndarray, input_dim: int, target_class: int, file_fold_seed: int) -> Tuple[Dict[str, float], List[Tuple[np.ndarray, np.ndarray]]]:
    skf = StratifiedKFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42 + file_fold_seed) 
    fold_metrics = {'accuracy': [], 'f1_weighted': []}
    all_validation_data = [] 
    
    for k_fold, (train_index, val_index) in enumerate(skf.split(X, y_binary)):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y_binary[train_index], y_binary[val_index]

        train_dataset = FeatureDataset(X_train, y_train, is_ovr_target=True)
        val_dataset = FeatureDataset(X_val, y_val, is_ovr_target=True)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        model = BinaryCNN(input_dim, NUM_BINARY_CLASSES).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(N_EPOCHS):
            model.train()
            for features, labels in train_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        model.eval()
        fold_preds = []
        fold_labels = []
        fold_scores = []
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                outputs = model(features)
                
                _, predicted = torch.max(outputs.data, 1)
                
                fold_preds.extend(predicted.cpu().numpy())
                fold_labels.extend(labels.cpu().numpy())
                fold_scores.extend(outputs[:, 1].cpu().numpy()) 

        acc = accuracy_score(fold_labels, fold_preds)
        f1_w = f1_score(fold_labels, fold_preds, average='weighted', zero_division=0) 
        
        fold_metrics['accuracy'].append(acc)
        fold_metrics['f1_weighted'].append(f1_w)
        all_validation_data.append((val_index, np.array(fold_scores)))
    
    mean_f1_weighted = np.mean(fold_metrics['f1_weighted'])
    
    mean_metrics = {
        'Target_Class': target_class,
        'Mean_Accuracy': np.mean(fold_metrics['accuracy']),
        'Std_Accuracy': np.std(fold_metrics['accuracy']),
        'Mean_F1_Weighted': mean_f1_weighted,
        'Std_F1_Weighted': np.std(fold_metrics['f1_weighted'])
    }
    
    return mean_metrics, all_validation_data

def run_ovr_training(df: pd.DataFrame, filename: str, file_index: int, feature_cols: List[str]) -> Tuple[float, np.ndarray]:
    sex_mapping = {'M': 0, 'F': 1}
    df_temp = df.copy()
    df_temp['Sex'] = df_temp['Sex'].map(sex_mapping)
    
    X_raw = df_temp[feature_cols].values 
    y_original = df_temp['Class'].values
    
    X = np.nan_to_num(X_raw, nan=0.0) 
    INPUT_DIM = X.shape[1]
    TOTAL_SAMPLES = len(X)
    
    aggregated_ovr_scores = np.zeros((TOTAL_SAMPLES, NUM_CLASSES)) 
    
    for target_class in TARGET_CLASSES:
        y_binary = np.where(y_original == target_class, 1, 0)
        results, validation_data = train_ovr_model_and_collect_scores(
            X, y_binary, INPUT_DIM, target_class, file_fold_seed=file_index + target_class
        )

        ovr_class_index = target_class - 1 
        for val_index, scores in validation_data:
            aggregated_ovr_scores[val_index, ovr_class_index] = scores

    final_predictions_1_to_5 = np.argmax(aggregated_ovr_scores, axis=1) + 1 
    true_labels_1_to_5 = y_original
    
    conf_matrix = confusion_matrix(
        true_labels_1_to_5, 
        final_predictions_1_to_5, 
        labels=TARGET_CLASSES
    )
    
    total_f1_weighted = f1_score(true_labels_1_to_5, final_predictions_1_to_5, average='weighted', zero_division=0)
    
    return total_f1_weighted, conf_matrix

# MATRIZ DE CONFUSÃO
def display_confusion_matrix(conf_matrix: np.ndarray, filename: str, save_dir: str = OUTPUT_DIR):
    print("\n" + "#"*70)
    print(f"MATRIZ DE CONFUSÃO (5x5) OVR - Agregação de K-Fold para: {filename}")
    print(f"Rótulos: [1, 2, 3, 4, 5]")
    print("#"*70)
    
    header = ["Classe Real ->"] + [f"Pred {c}" for c in TARGET_CLASSES]
    print("{:<15}".format(""), end="")
    print(" ".join(["{:>8}".format(h) for h in header[1:]]))
    print("-" * (15 + 8 * 5))
    for i in range(5):
        row_label = f"Classe {TARGET_CLASSES[i]} |"
        print("{:<15}".format(row_label), end="")
        for j in range(5):
            print("{:>8}".format(conf_matrix[i, j]), end="")
        print()
    print("#"*70)
    
    try:
        plt.figure(figsize=(7, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                    xticklabels=TARGET_CLASSES, yticklabels=TARGET_CLASSES)
        plt.title(f'Matriz de Confusão - {filename}')
        plt.xlabel('Predito')
        plt.ylabel('Real')
        plt.tight_layout()
        
        save_path = os.path.join(save_dir, f'confusion_matrix_{filename.replace(".csv", "")}.png')
        plt.savefig(save_path, dpi=300)
        plt.close()
        
        print(f"Matriz de confusão salva em: {save_path}")
    except Exception as e:
         print(f"AVISO: Não foi possível salvar a matriz de confusão gráfica. Erro: {e}")

# SELEÇÃO SEQUENCIAL DE FEATURES (SFS Forward)
def sequential_feature_selection(df: pd.DataFrame, filename: str, file_index: int):
    
    print(f"\n{'='*70}\nINICIANDO SELEÇÃO SEQUENCIAL DE FEATURES (SFS) para: {filename}\n{'='*70}")
    
    initial_cols = ['Age', 'Sex']
    initial_cols.extend(get_grouped_columns('Fo_mean_Hz'))
    
    current_features = initial_cols 
    remaining_groups = FEATURE_GROUPS
    
    print(f"Teste Inicial (Base): {current_features} ({len(current_features)} colunas)")
    best_score, best_conf_matrix = run_ovr_training(df, filename, file_index, current_features)
    print(f"F1-Weighted Inicial: {best_score:.4f}")
    
    print("\nSFS FORWARD")
    
    history = [
        {'step': 0, 'features': list(current_features), 'score': best_score, 'action': 'START'}
    ]

    step = 1
    while remaining_groups:
        
        best_gain = -1.0
        best_group_to_add = None
        best_temp_score = best_score
        
        for group_to_add in remaining_groups:
            temp_features = current_features + group_to_add
            temp_score, _ = run_ovr_training(df, filename, file_index, temp_features)
            gain = temp_score - best_score
            group_name = group_to_add[0].split('_')[0]
            print(f"Step {step} - Testando grupo '{group_name}': F1={temp_score:.4f} (Ganho: {gain:.4f})")
            
            if gain > best_gain:
                best_gain = gain
                best_group_to_add = group_to_add
                best_temp_score = temp_score
        
        if best_gain > 1e-4:
            current_features.extend(best_group_to_add)
            best_score = best_temp_score
            remaining_groups.remove(best_group_to_add)
            _, best_conf_matrix = run_ovr_training(df, filename, file_index, current_features) 
            
            group_name = best_group_to_add[0].split('_')[0]
            print(f"\nPasso {step} (ADD): Grupo '{group_name}' adicionado.")
            print(f"Novo Melhor F1-Weighted: {best_score:.4f} ({len(current_features)} colunas)\n")
            
            history.append({
                'step': step, 
                'features': list(current_features), 
                'score': best_score, 
                'action': f'ADD: {group_name}'
            })
            
            step += 1
        else:
            print(f"\nPasso {step} (STOP): Nenhuma melhoria significativa ({best_gain:.4f}). Fim do SFS Forward.")
            break

    print("\n" + "="*70)
    print(f"RESULTADO FINAL DO SFS PARA {filename}")
    print(f"Melhor Conjunto de Features ({len(current_features)}): {current_features}")
    print(f"Melhor F1-Weighted Agregado: {best_score:.4f}")
    print("="*70)
    
    display_confusion_matrix(best_conf_matrix, f"SFS_Final_{filename}", save_dir=OUTPUT_DIR)

    history_df = pd.DataFrame(history)
    history_path = os.path.join(OUTPUT_DIR, f'sfs_history_{filename.replace(".csv", "")}.csv')
    history_df.to_csv(history_path, index=False)
    print(f"Histórico do SFS salvo em: {history_path}")

# EXECUÇÃO
try:
    filepath = os.path.join(OUTPUT_DIR, INPUT_FILE_FOR_SFS)

    if not os.path.exists(filepath):
        print(f"ERRO: Arquivo '{INPUT_FILE_FOR_SFS}' não encontrado em '{OUTPUT_DIR}'")
    else:
        df = pd.read_csv(filepath)
        sequential_feature_selection(df, INPUT_FILE_FOR_SFS, file_index=0)

except Exception as e:
    print(f"Erro fatal ao carregar ou processar o arquivo {INPUT_FILE_FOR_SFS}: {e}", file=sys.stderr)



INICIANDO SELEÇÃO SEQUENCIAL DE FEATURES (SFS) para: features_z_score.csv
1. Teste Inicial (Base): ['Age', 'Sex', 'Fo_mean_Hz_phonationA', 'Fo_mean_Hz_phonationE', 'Fo_mean_Hz_phonationI', 'Fo_mean_Hz_phonationO', 'Fo_mean_Hz_phonationU'] (7 colunas)
-> F1-Weighted Inicial: 0.2662

SFS FORWARD
Step 1 - Testando grupo 'Fhi': F1=0.3260 (Ganho: 0.0598)
Step 1 - Testando grupo 'Flo': F1=0.3449 (Ganho: 0.0788)
Step 1 - Testando grupo 'F0': F1=0.2993 (Ganho: 0.0331)
Step 1 - Testando grupo 'Jitter': F1=0.3385 (Ganho: 0.0723)
Step 1 - Testando grupo 'Jitter': F1=0.3017 (Ganho: 0.0356)
Step 1 - Testando grupo 'RAP': F1=0.3136 (Ganho: 0.0474)
Step 1 - Testando grupo 'PPQ': F1=0.3023 (Ganho: 0.0362)
Step 1 - Testando grupo 'DDP': F1=0.3114 (Ganho: 0.0452)
Step 1 - Testando grupo 'Shimmer': F1=0.3478 (Ganho: 0.0817)
Step 1 - Testando grupo 'Shimmer': F1=0.3258 (Ganho: 0.0597)
Step 1 - Testando grupo 'Shimmer': F1=0.3447 (Ganho: 0.0785)
Step 1 - Testando grupo 'Shimmer': F1=0.3502 (Ganho: 0.0841)